# Grid search for hyperparameter tunning (includes gate speed and grip strength)

# Import and setup


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import display, clear_output
import time
import itertools

from mop_model import MoPModel as MoP_raw, MoPConfig


# Trainning and evaluation functions

In [26]:
def train_and_evaluate_trial_mop(params, X_train, y_train, X_val, y_val):
    """
    Trains and evaluates a single trial of the MoP model with a given set of hyperparameters.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # --- Model Configuration ---
    config = MoPConfig(
        input_dim=X_train.shape[1],
        output_dim=2,
        intermediate_dim=params['intermediate_dim'],
        layers=params['layers']
    )
    model = MoP_raw(config).to(device)
    
    # --- Data Preparation ---
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_val_tensor = torch.LongTensor(y_val).to(device)

    # --- Training with Early Stopping ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    max_epochs = 75

    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        # MoP expects a sequence dimension
        y_pred, usage_losses, entropy_loss = model(X_train_tensor.unsqueeze(1))
        y_pred = y_pred.squeeze(1)
        
        # MoP auxiliary losses can be added here for more robust training
        class_loss = criterion(y_pred, y_train_tensor)
        total_loss = class_loss # For simplicity, we only use class_loss for early stopping
        
        total_loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            y_val_pred, _, _ = model(X_val_tensor.unsqueeze(1))
            val_loss = criterion(y_val_pred.squeeze(1), y_val_tensor)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            break

    # --- Final Evaluation on Validation Set ---
    model.eval()
    with torch.no_grad():
        y_pred_tensor, _, _ = model(X_val_tensor.unsqueeze(1))
        y_pred_tensor = y_pred_tensor.squeeze(1)
        probas = nn.functional.softmax(y_pred_tensor, dim=1)
        _, predicted = torch.max(probas, 1)
        
        y_true = y_val_tensor.cpu().numpy()
        y_pred = predicted.cpu().numpy()
        y_score = probas[:, 1].cpu().numpy()
        
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0).get('1', {})

        return {
            'intermediate_dim': params['intermediate_dim'],
            'layers': str(params['layers']), # Convert list to string for display
            'val_roc_auc': roc_auc_score(y_true, y_score),
            'val_accuracy': accuracy_score(y_true, y_pred),
            'val_f1_score': report.get('f1-score', 0),
            'val_precision': report.get('precision', 0),
            'val_recall': report.get('recall', 0),
            'val_cohen_kappa': cohen_kappa_score(y_true, y_pred)
        }


# Data Loading and Hyperparameter definition

In [ ]:
# --- 1. Load and Prepare Data ---
try:
    print("🔹 Loading and preparing data...")
    df = pd.read_csv('input_mop_rfe_noninvasive.csv', low_memory=False)
    X = df.drop(columns=['Dementia Status'])
    y = df['Dementia Status']
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Data loaded successfully. Train shape: {X_train.shape}")
except FileNotFoundError:
    print("\n⚠️ ERROR: 'input_data.csv' not found. Please run the feature extractor first.")
    # In a notebook, you might want to stop execution here if the file is not found.

# --- 2. Define the Hyperparameter Grid for MoP ---
param_grid = {
    'intermediate_dim': [32, 64, 96, 128],
    'layers': [
        ["0,8,16", "0,8,16", "0,8,16"],      # Three simple layers
        ["0,16,32", "0,16,32", "0,16,32"],    # Three more complex layers
        ["0,8,16", "0,16,32", "0,8,16"],      # A mix of simple and complex layers
        ["0,16,32", "0,8,16", "0,16,32"]       # Another mix
    ]
}

# Create all possible combinations
valid_params = [
    {'intermediate_dim': p[0], 'layers': p[1]}
    for p in itertools.product(param_grid['intermediate_dim'], param_grid['layers'])
]

print(f"\n🔹 Starting Grid Search for MoP. Total combinations to test: {len(valid_params)}")


🔹 Loading and preparing data...
Data loaded successfully. Train shape: (1297, 24)

🔹 Starting Grid Search for MoP. Total combinations to test: 16


# Run grid search and final evaluation

In [ ]:
# --- 3. Run the Grid Search for MoP ---
results = []
for i, params in enumerate(valid_params):
    print(f"\n--- Testing Combination {i+1}/{len(valid_params)} ---")
    print(f"Parameters: {params}")
    
    result = train_and_evaluate_trial_mop(params, X_train_scaled, y_train.values, X_val_scaled, y_val.values)
    results.append(result)
    
    # Live update of results
    clear_output(wait=True)
    # --- CHANGE: Sort by validation accuracy ---
    results_df = pd.DataFrame(results).sort_values('val_accuracy', ascending=False)
    print("✅ Intermediate Tuning Results for MoP (Validation Set):")
    display(results_df)

# --- 4. Final Evaluation on Test Set ---
print("\n\n" + "="*40 + "\n✅ FINAL MoP EVALUATION ON HELD-OUT TEST SET\n" + "="*40)

# Get the best parameters from the grid search
best_params = results_df.iloc[0].to_dict()
# --- CHANGE: Update print statement to reflect accuracy ---
print("🏆 Best MoP Hyperparameters found (based on validation Accuracy):")
print(f"  - Intermediate Dimension: {int(best_params['intermediate_dim'])}")
print(f"  - Layer Structure: {best_params['layers']}")

# Combine training and validation data
X_train_val = np.concatenate((X_train_scaled, X_val_scaled), axis=0)
y_train_val = np.concatenate((y_train.values, y_val.values), axis=0)

print("\nRetraining the best MoP model on combined Train+Validation data...")

# Create the final model with the best parameters
final_config = MoPConfig(
    input_dim=X_train_val.shape[1],
    output_dim=2,
    intermediate_dim=int(best_params['intermediate_dim']),
    layers=eval(best_params['layers']) # Use eval to convert string back to list
)
final_model = MoP_raw(final_config)

# Train the final model and evaluate on the test set
# We reuse the same function, but the validation set is now the test set
final_results = train_and_evaluate_trial_mop(
    {'intermediate_dim': int(best_params['intermediate_dim']), 'layers': eval(best_params['layers'])},
    X_train_val, y_train_val, X_test_scaled, y_test.values
)

print("\n--- Final MoP Performance on Test Set ---")
print(f"  - Accuracy:    {final_results['val_accuracy']:.4f}")
print(f"  - ROC-AUC:     {final_results['val_roc_auc']:.4f}")
print(f"  - F1-Score:    {final_results['val_f1_score']:.4f}")
print(f"  - Precision:   {final_results['val_precision']:.4f}")
print(f"  - Recall:      {final_results['val_recall']:.4f}")
print(f"  - Cohen Kappa: {final_results['val_cohen_kappa']:.4f}")

✅ Intermediate Tuning Results for MoP (Validation Set):


,intermediate_dim,layers,val_roc_auc,val_accuracy,val_f1_score,val_precision,val_recall,val_cohen_kappa
6,64,"['0,8,16', '0,16,32', '0,8,16']",0.767899,0.715827,0.744337,0.684524,0.815603,0.429943
14,128,"['0,8,16', '0,16,32', '0,8,16']",0.768960,0.712230,0.736842,0.687117,0.794326,0.423027
7,64,"['0,16,32', '0,8,16', '0,16,32']",0.758788,0.708633,0.734426,0.682927,0.794326,0.415754
8,96,"['0,8,16', '0,8,16', '0,8,16']",0.754620,0.708633,0.739550,0.676471,0.815603,0.415390
12,128,"['0,8,16', '0,8,16', '0,8,16']",0.762955,0.708633,0.736156,0.680723,0.801418,0.415633
0,32,"['0,8,16', '0,8,16', '0,8,16']",0.759694,0.705036,0.733766,0.676647,0.801418,0.408357
1,32,"['0,16,32', '0,16,32', '0,16,32']",0.761764,0.705036,0.746914,0.661202,0.858156,0.407373
9,96,"['0,16,32', '0,16,32', '0,16,32']",0.765388,0.705036,0.733766,0.676647,0.801418,0.408357
2,32,"['0,8,16', '0,16,32', '0,8,16']",0.759435,0.701439,0.733119,0.670588,0.808511,0.400955
3,32,"['0,16,32', '0,8,16', '0,16,32']",0.753688,0.701439,0.718644,0.688312,0.751773,0.401949




✅ FINAL MoP EVALUATION ON HELD-OUT TEST SET
🏆 Best MoP Hyperparameters found (based on validation Accuracy):
  - Intermediate Dimension: 64
  - Layer Structure: ['0,8,16', '0,16,32', '0,8,16']

Retraining the best MoP model on combined Train+Validation data...

--- Final MoP Performance on Test Set ---
  - Accuracy:    0.7590
  - ROC-AUC:     0.8145
  - F1-Score:    0.7744
  - Precision:   0.7325
  - Recall:      0.8214
  - Cohen Kappa: 0.5175
